Importing all Libraries

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns   # for detecting outliers
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
from sklearn import linear_model
from sklearn import metrics
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
import collections

In [2]:
SEED = 222
np.random.seed(SEED)

In [3]:
train = pd.read_csv('equip_failures_training_set.csv', na_values='na')
test = pd.read_csv('equip_failures_test_set.csv', na_values='na')
sensor_data = train
#sensor_data.head()
#sensor_data.describe()

# Data Processing 

NAs: columns with more than 65% are not considered.


In [4]:
sensor_data = sensor_data.drop(['sensor2_measure','sensor38_measure','sensor39_measure', 'sensor40_measure','sensor41_measure','sensor42_measure','sensor43_measure','sensor68_measure','sensor68_measure'], axis=1)

In [5]:
X = sensor_data.drop(['id', 'target'], axis=1)
y = sensor_data['target']

First a train-test split is done after which imputation of 'na' values with median of train split is done.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=222)

We consider weights for loss during training of the LightGBM model to tackle the class imbalance issue.
Class-imbalance ratio = 1.67:98.33

In [7]:
weights = np.asarray(y_train)
weights[weights==1] = 6
weights[weights==0] = 1

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=222)
#X_test, X_cv, y_test, y_cv = train_test_split(X_test, y_test, test_size=0.5, random_state=222)

In [9]:
for col in X_train.columns:
    if col != ('id' or 'target'):
        X_train[col]=X_train[col].fillna(X_train[col].median())

C:\Users\Ganatma\Anaconda3\envs\tamu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
for col in X_test.columns:
    if col != ('id' or 'target'):
        X_test[col]=X_test[col].fillna(X_train[col].median())

C:\Users\Ganatma\Anaconda3\envs\tamu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#for col in X_cv.columns:
#    if col != ('id' or 'target'):
#        X_cv[col]=X_cv[col].fillna(X_train[col].median())

In [28]:
train_data=lgb.Dataset(X_train,label=y_train,weight = weights)
#define parameters

params = {'learning_rate':0.12,'num_leaves': 200, 'objective': 'binary'}
model= lgb.train(params, train_data, 100) 
y_pred=model.predict(X_test)
y_pred = np.around(y_pred)

In [29]:
metrics.accuracy_score(y_test, y_pred)

0.9940555555555556

In [30]:
metrics.f1_score(y_test, y_pred)

0.8043875685557588

In [31]:
xtest = test.drop(['id','sensor2_measure','sensor38_measure','sensor39_measure', 'sensor40_measure','sensor41_measure','sensor42_measure','sensor43_measure','sensor68_measure','sensor68_measure'], axis=1)
data = xtest

for col in data.columns:
    if col != ('id' or 'target'):
        data[col]=data[col].fillna(X_train[col].median())

In [32]:
pred = np.around(model.predict(data))

In [33]:
dataset = pd.DataFrame({'id': range(1,len(pred)+1), 'target': pred})

In [36]:
dataset.to_csv(r'D:\TAMU\Datathon\final-submission.csv')

Our code can be deployed over any dataset by reading it into the test_dataset which takes care of all column deletions